In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Preparación de datos

In [46]:
def data_to_df(path):
    df = pd.read_csv(path, sep = ',')
    df = df.melt(id_vars = ('Province/State','Country/Region','Lat','Long'), var_name='Date', value_name='Cases')
    return df


def consolidate_data(c_path, d_path, r_path):
    'parameters: c_path = path to confirmed cases table, d_path = path to deaths cases table, r_path = path to recovered cases table'
    df1 = data_to_df(c_path)
    df2 = data_to_df(d_path)
    df3 = data_to_df(r_path)

    df1['Status'] = 'Confirmed'
    df2['Status'] = 'Deaths'
    df3['Status'] = 'Recovered'

    data = df1.append(df2)
    data = data.append(df3)
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
    data['Year-month'] = pd.to_datetime(data['Date'], errors='coerce').dt.to_period('M')

    return data

In [47]:
confirmed_path = 'data_sources/time_series_covid19_confirmed_global.csv'
deaths_path = 'data_sources/time_series_covid19_deaths_global.csv'
recovered_path = 'data_sources/time_series_covid19_recovered_global.csv'

data = consolidate_data(confirmed_path, deaths_path, recovered_path)
data

,Province/State,Country/Region,Lat,Long,Date,Cases,Status,Year-month
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,Confirmed,2020-01
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,Confirmed,2020-01
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,Confirmed,2020-01
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,Confirmed,2020-01
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,Confirmed,2020-01
...,...,...,...,...,...,...,...,...
178340,NaN,Vietnam,14.058324,108.277199,2021-11-24,0,Recovered,2021-11
178341,NaN,West Bank and Gaza,31.952200,35.233200,2021-11-24,0,Recovered,2021-11
178342,NaN,Yemen,15.552727,48.516388,2021-11-24,0,Recovered,2021-11
178343,NaN,Zambia,-13.133897,27.849332,2021-11-24,0,Recovered,2021-11


### Datos resumidos por país y fecha según el status

In [48]:
df = data[data.Status == 'Confirmed']
df_daily = df.groupby(['Date','Country/Region'], as_index=False).sum()
df_daily = df_daily[['Date','Country/Region','Cases']]
df_daily.head()

,Date,Country/Region,Cases
0,2020-01-22,Afghanistan,0
1,2020-01-22,Albania,0
2,2020-01-22,Algeria,0
3,2020-01-22,Andorra,0
4,2020-01-22,Angola,0


### Datos resumidos por Año-mes, país, status

In [49]:
data_summary = data[['Year-month', 'Country/Region', 'Status','Cases']]
data_summary = data_summary.groupby(['Year-month','Country/Region','Status'], as_index=False).sum()
data_summary.head()

,Year-month,Country/Region,Status,Cases
0,2020-01,Afghanistan,Confirmed,0
1,2020-01,Afghanistan,Deaths,0
2,2020-01,Afghanistan,Recovered,0
3,2020-01,Albania,Confirmed,0
4,2020-01,Albania,Deaths,0


## Obtener valores de cada día

In [50]:
# lista de paises y status
paises = np.unique(data['Country/Region'])
estados = np.unique(data['Status'])
data_real = pd.DataFrame()

# ciclo
for pais in paises:
    for estado in estados:
        # filtrar el dataframe original por pais y estado, remover los índices
        df = data[data['Country/Region'] == pais]
        df = df[df['Status'] == estado]
        df = df.sort_values(by=['Status','Province/State','Date'])
        df.reset_index(inplace=True)

        #calcular un dataframe con lag
        Previous_Cases = df[['Date','Status','Cases']]
        Previous_Cases = Previous_Cases.shift(periods=1)
        Previous_Cases.reset_index(inplace=True)

        #unir ambos dataframe por índice y calcular el valor real de cada día
        df = df.merge(Previous_Cases, left_index=True, right_index=True)
        df['Cases'] =  df['Cases_x'] - df['Cases_y']

        # convertir valores negativos a cero para los días sin datos
        for i in range(len(df)):
            df['Cases'][i] = 0 if df.iloc[i]['Cases'] <0 else df['Cases'][i]
        
        data_real = data_real.append(df)

#renombrar columnas
data_real = data_real[['Province/State','Country/Region','Lat','Long','Date_x','Status_x','Year-month','Cases']]
data_real = data_real.rename(columns={'Date_x':'Date','Status_x':'Status'})


C:\Users\eddso\anaconda3\envs\env_algoritmos\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyboardInterrupt: 

In [ ]:
data_real

,Province/State,Country/Region,Lat,Long,Date,Status,Year-month,Cases
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,Confirmed,2020-01,NaN
1,NaN,Afghanistan,33.939110,67.709953,2020-01-23,Confirmed,2020-01,0.0
2,NaN,Afghanistan,33.939110,67.709953,2020-01-24,Confirmed,2020-01,0.0
3,NaN,Afghanistan,33.939110,67.709953,2020-01-25,Confirmed,2020-01,0.0
4,NaN,Afghanistan,33.939110,67.709953,2020-01-26,Confirmed,2020-01,0.0
...,...,...,...,...,...,...,...,...
668,NaN,Zimbabwe,-19.015438,29.154857,2021-11-20,Recovered,2021-11,0.0
669,NaN,Zimbabwe,-19.015438,29.154857,2021-11-21,Recovered,2021-11,0.0
670,NaN,Zimbabwe,-19.015438,29.154857,2021-11-22,Recovered,2021-11,0.0
671,NaN,Zimbabwe,-19.015438,29.154857,2021-11-23,Recovered,2021-11,0.0


In [ ]:
datos = pd.read_csv('data_sources/data.csv')
datos

C:\Users\eddso\anaconda3\envs\env_algoritmos\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Province/State,Country/Region,Lat,Long,Date,Status,Year-month,Cases
0,0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,Confirmed,2020-01,NaN
1,1,NaN,Afghanistan,33.939110,67.709953,2020-01-23,Confirmed,2020-01,0.0
2,2,NaN,Afghanistan,33.939110,67.709953,2020-01-24,Confirmed,2020-01,0.0
3,3,NaN,Afghanistan,33.939110,67.709953,2020-01-25,Confirmed,2020-01,0.0
4,4,NaN,Afghanistan,33.939110,67.709953,2020-01-26,Confirmed,2020-01,0.0
...,...,...,...,...,...,...,...,...,...
555220,668,NaN,Zimbabwe,-19.015438,29.154857,2021-11-20,Recovered,2021-11,0.0
555221,669,NaN,Zimbabwe,-19.015438,29.154857,2021-11-21,Recovered,2021-11,0.0
555222,670,NaN,Zimbabwe,-19.015438,29.154857,2021-11-22,Recovered,2021-11,0.0
555223,671,NaN,Zimbabwe,-19.015438,29.154857,2021-11-23,Recovered,2021-11,0.0


In [119]:
data2 = pd.pivot_table(datos,index=['Country/Region','Lat','Long','Date','Year-month'],columns='Status', values='Cases', aggfunc=np.sum, observed=True)
#data2.reset_index(inplace=True)
#data2['Confirmed'] = data2['Confirmed'].fillna(0)
#data2['Deaths'] = data2['Deaths'].fillna(0)
#data2['Recovered'] = data2['Recovered'].fillna(0)
#data2 = data2.groupby(by=['Country/Region','Year-month']).sum()
#data2['Mortality Rate'] = data2['Deaths']/data2['Confirmed']
#data2.reset_index(inplace=True)
#data2 = data2[['Country/Region','Year-month','Mortality Rate']]
#data2 = data2.dropna(subset=['Mortality Rate'])
data2


Status                                                     Confirmed  Deaths  \
Country/Region Lat        Long      Date       Year-month                      
Afghanistan     33.939110 67.709953 2020-01-22 2020-01           0.0     0.0   
                                    2020-01-23 2020-01           0.0     0.0   
                                    2020-01-24 2020-01           0.0     0.0   
                                    2020-01-25 2020-01           0.0     0.0   
                                    2020-01-26 2020-01           0.0     0.0   
...                                                              ...     ...   
Zimbabwe       -19.015438 29.154857 2021-11-20 2021-11          22.0     0.0   
                                    2021-11-21 2021-11          32.0     0.0   
                                    2021-11-22 2021-11          27.0     0.0   
                                    2021-11-23 2021-11           0.0     0.0   
                                    2021-11-24 2021-11          73.0     4.0   

Status                                                     Recovered  
Country/Region Lat        Long      Date       Year-month             
Afghanistan     33.939110 67.709953 2020-01-22 2020-01           0.0  
                                    2020-01-23 2020-01           0.0  
                                    2020-01-24 2020-01           0.0  
                                    2020-01-25 2020-01           0.0  
                                    2020-01-26 2020-01           0.0  
...                                                              ...  
Zimbabwe       -19.015438 29.154857 2021-11-20 2021-11           0.0  
                                    2021-11-21 2021-11           0.0  
                                    2021-11-22 2021-11           0.0  
                                    2021-11-23 2021-11           0.0  
                                    2021-11-24 2021-11           0.0  

[189786 rows x 3 columns]

In [118]:
data2.to_csv('data2.csv')